In [ ]:
%load_ext autoreload
%autoreload 2

import argparse
import json, os
import numpy as np

from bert_multiple_choice.data_module import DataCollatorForMultipleChoice
from tqdm import tqdm
from transformers import (
    AutoModelForMultipleChoice, 
    TrainingArguments, 
    Trainer, 
    AutoTokenizer
)

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("multiple_choice/experiments")
model = AutoModelForMultipleChoice.from_pretrained("multiple_choice/experiments")


In [ ]:
history = "hello , how are you doing tonight ? i am great . i just got back from the club . i would rather eat chocolate cake during this season . i went to club chino . what show are you watching ? do you live in a house or apartment ? i love those shows . i am really craving cake . it matters because i have a sweet tooth . my family lives in alaska . it is freezing down there ."
choice0 = "i just bought a brand new house. i like to dance at the club. i run a dog obedience school. i have a big sweet tooth. i like taking and posting selkies."
choice1 = "i love to meet new people. i have a turtle named timothy. my favorite sport is ultimate frisbee. my parents are living in bora bora. autumn is my favorite season."
choice2 = "i just bought a brand new house. i like to dance at the club. i run a dog obedience school. i like taking and posting selkies."

labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([history, history, history], [choice2, choice1, choice2], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=None)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

logits = logits.detach().cpu().numpy()

print(np.argmax(logits, axis=-1))